In [20]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-12-16 21:21:26.948851


In [21]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Collections

In [3]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parse Artists (Parallel)

In [41]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #arts.createArtistModValMetadata(modVal=modVal)
    #arts.createArtistAlbumModValMetadata(modVal=modVal)
    
def parseArtistsParallel(nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    #result = pool.map_async(parseArtists, [68,53,49,48])

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
#arts.parseArtistModValFiles(68, force=True)

In [43]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Tue Dec 17, 2019 00:24:12 for Parsing All Artists
Current Time is Tue Dec 17, 2019 00:24:12 for Parallel Artist Parsing
Parsing Artist Files For ModVal 18
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 9
  ===> Found 5952 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 1
  ===> Found 5936 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 10
  ===> Found 6130 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 19
  ===> Found 6021 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 11
  ===> Found 6014 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 20
  ===> Found 6183 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 2
  ===> Found 6087 files. There are 1 new files to parse.
  ===> Found 6050 files, but there are no new files 

Parsing Artist Files For ModVal 56
  ===> Found 6059 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 57
  ===> Found 6012 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 58
  --> This file is 15.1MB.
Saving 5984 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/46-Metadata.p
  --> This file is 356.6kB.
Saving 5984 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/46-MediaMetadata.p
  --> This file is 4.8MB.
Parsing Artist Files For ModVal 47
  ===> Found 6021 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 59
  ===> Found 5833 files. There are 1 new files to parse.
  ===> Found 6141 files. There are 1 new files to parse.
  --> This file is 11.1MB.
Saving 6134 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/32-Metadata.p
  --> This file is 369.2kB.
Saving 6134 new artist IDs media data to /Volumes/Music/Dis

Saving 6047 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/82-Metadata.p
  --> This file is 358.2kB.
Saving 6047 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/82-MediaMetadata.p
  ===> Found 5834 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 91
  --> This file is 3.4MB.
Parsing Artist Files For ModVal 83
Saving 1 new artist IDs to /Volumes/Music/Discog/artists-db/76-DB.p
  ===> Found 5995 files. There are 1 new files to parse.
  ===> Found 6115 files. There are 1 new files to parse.
Saving 1 new artist IDs to /Volumes/Music/Discog/artists-db/91-DB.p
Saving 1 new artist IDs to /Volumes/Music/Discog/artists-db/83-DB.p
  --> This file is 11.0MB.
Saving 6076 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/76-Metadata.p
  --> This file is 362.4kB.
Saving 6076 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/76-MediaMetadata.p
  --> This file is 3.6MB.
Parsing

# Albums

In [14]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
albs.createAlbumModValMetadata(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Saving 4171 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p
  --> This file is 962.0kB.
Saving 4171 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/0-ArtistAlbums.p
  --> This file is 2.1MB.


In [15]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [16]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Dec 15, 2019 16:46:31 for Parsing All Albums
Current Time is Sun Dec 15, 2019 16:46:31 for Parallel Album Parsing
Current Time is Sun Dec 15, 2019 16:46:31 for Parsing Album Files For ModVal NAN
Current Time is Sun Dec 15, 2019 16:46:31 for Parsing Album Files For ModVal 8
Current Time is Sun Dec 15, 2019 16:46:31 for Parsing Album Files For ModVal 17
  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
  Looking for artist albums in /Volumes/Music/Discog/albums/8
  Looking for artist albums in /Volumes/Music/Discog/albums/17
Current Time is Sun Dec 15, 2019 16:46:31 for Processing ['NAN', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73

Saving 495 new album IDs to /Volumes/Music/Discog/albums-db/17-DB.p
  --> This file is 9.1MB.
Saving 4141 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/8-ArtistMetadata.p
  --> This file is 794.4kB.
Saving 4141 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/8-ArtistAlbums.p
  --> This file is 10.3MB.
Saving 4176 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/17-ArtistMetadata.p
  --> This file is 1.3MB.
Current Time is Sun Dec 15, 2019 16:49:36 for Done with Parsing Album Files For ModVal 8
Process [{0}] took 3.1 minutes.
Current Time is Sun Dec 15, 2019 16:49:36 for Parsing Album Files For ModVal 9
  Looking for artist albums in /Volumes/Music/Discog/albums/9
  --> This file is 807.9kB.
Saving 4176 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p
  --> This file is 1.5MB.
Current Time is Sun Dec 15, 2019 16:49:42 for Done with Parsing Album Files For ModVal 17
Process [{0}] took 3.2 minutes.
Current Time is Sun Dec 15, 2

    Found 1   albums (411  ) in 3000/3271	/Volumes/Music/Discog/albums/18/7399218
    Found 1   albums (531  ) in 5600/5862	/Volumes/Music/Discog/albums/9/835209
    Found 3   albums (434  ) in 3100/3271	/Volumes/Music/Discog/albums/18/83518
    Found 6   albums (552  ) in 5700/5862	/Volumes/Music/Discog/albums/9/885309
    Found 4   albums (566  ) in 5800/5862	/Volumes/Music/Discog/albums/9/956009
    Found 1   albums (575  ) in 5861/5862	/Volumes/Music/Discog/albums/9/999609
Saving 575 new album IDs to /Volumes/Music/Discog/albums-db/9-DB.p
    Found 1   albums (474  ) in 3200/3271	/Volumes/Music/Discog/albums/18/920718
    Found 10  albums (488  ) in 3270/3271	/Volumes/Music/Discog/albums/18/999718
Saving 488 new album IDs to /Volumes/Music/Discog/albums-db/18-DB.p
  --> This file is 11.3MB.
Saving 5862 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/9-ArtistMetadata.p
  --> This file is 983.4kB.
Saving 5862 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/9-Art

    Found 0   albums (3133 ) in 700/4843	/Volumes/Music/Discog/albums/10/1528110
    Found 1   albums (61   ) in 300/3227	/Volumes/Music/Discog/albums/20/1349520
    Found 18  albums (75   ) in 400/3227	/Volumes/Music/Discog/albums/20/146420
    Found 2   albums (89   ) in 500/3227	/Volumes/Music/Discog/albums/20/160820
    Found 1   albums (115  ) in 600/3227	/Volumes/Music/Discog/albums/20/1728120
    Found 1   albums (123  ) in 700/3227	/Volumes/Music/Discog/albums/20/1882320
    Found 0   albums (127  ) in 800/3227	/Volumes/Music/Discog/albums/20/2020
    Found 13  albums (3424 ) in 800/4843	/Volumes/Music/Discog/albums/10/1593910
    Found 0   albums (141  ) in 900/3227	/Volumes/Music/Discog/albums/20/2146020
    Found 1   albums (155  ) in 1000/3227	/Volumes/Music/Discog/albums/20/2303620
    Found 6   albums (170  ) in 1100/3227	/Volumes/Music/Discog/albums/20/2450420
    Found 1   albums (179  ) in 1200/3227	/Volumes/Music/Discog/albums/20/2621720
    Found 1   albums (192  ) i

    Found 0   albums (36   ) in 700/5903	/Volumes/Music/Discog/albums/22/1544022
    Found 1   albums (39   ) in 800/5903	/Volumes/Music/Discog/albums/22/16222
    Found 2   albums (48   ) in 900/5903	/Volumes/Music/Discog/albums/22/1686222
    Found 1   albums (54   ) in 1000/5903	/Volumes/Music/Discog/albums/22/1760922
    Found 1   albums (54   ) in 1100/5903	/Volumes/Music/Discog/albums/22/1835022
    Found 1   albums (58   ) in 1200/5903	/Volumes/Music/Discog/albums/22/1904822
    Found 1   albums (73   ) in 1300/5903	/Volumes/Music/Discog/albums/22/1984822
    Found 0   albums (78   ) in 1400/5903	/Volumes/Music/Discog/albums/22/2058322
    Found 2   albums (83   ) in 1500/5903	/Volumes/Music/Discog/albums/22/213722
    Found 1   albums (87   ) in 1600/5903	/Volumes/Music/Discog/albums/22/2212122
    Found 2   albums (89   ) in 1700/5903	/Volumes/Music/Discog/albums/22/2301422
    Found 2   albums (91   ) in 1800/5903	/Volumes/Music/Discog/albums/22/239222
    Found 0   albums (9

...

Current Time is Sun Dec 15, 2019 19:10:26 for Parsing Album Files For ModVal 88
  Looking for artist albums in /Volumes/Music/Discog/albums/88
  --> This file is 13.4MB.
Saving 5751 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/90-ArtistMetadata.p
  --> This file is 1.0MB.
Saving 5751 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/90-ArtistAlbums.p
  --> This file is 2.1MB.
Current Time is Sun Dec 15, 2019 19:10:35 for Done with Parsing Album Files For ModVal 90
Process [{0}] took 2.6 minutes.
Current Time is Sun Dec 15, 2019 19:10:35 for Parsing Album Files For ModVal 91
  Looking for artist albums in /Volumes/Music/Discog/albums/91
  Found 5890 artist directories
    Found 1   albums (0    ) in 0/5890	/Volumes/Music/Discog/albums/88/1000688
    Found 1   albums (6    ) in 100/5890	/Volumes/Music/Discog/albums/88/1074688
    Found 0   albums (16   ) in 200/5890	/Volumes/Music/Discog/albums/88/115688
    Found 8   albums (24   ) in 300/5890	/Volumes/Music/Disco

    Found 9   albums (395  ) in 3500/4146	/Volumes/Music/Discog/albums/91/6091
    Found 3   albums (384  ) in 5700/5890	/Volumes/Music/Discog/albums/88/86788
    Found 1   albums (404  ) in 3600/4146	/Volumes/Music/Discog/albums/91/6433791
    Found 2   albums (403  ) in 5800/5890	/Volumes/Music/Discog/albums/88/92788
    Found 2   albums (412  ) in 3700/4146	/Volumes/Music/Discog/albums/91/68291
    Found 2   albums (409  ) in 5889/5890	/Volumes/Music/Discog/albums/88/999988
Saving 409 new album IDs to /Volumes/Music/Discog/albums-db/88-DB.p
    Found 0   albums (422  ) in 3800/4146	/Volumes/Music/Discog/albums/91/726591
    Found 3   albums (455  ) in 3900/4146	/Volumes/Music/Discog/albums/91/799891
    Found 1   albums (483  ) in 4000/4146	/Volumes/Music/Discog/albums/91/877691
    Found 1   albums (504  ) in 4100/4146	/Volumes/Music/Discog/albums/91/961691
    Found 1   albums (510  ) in 4145/4146	/Volumes/Music/Discog/albums/91/9991
Saving 510 new album IDs to /Volumes/Music/Disc

    Found 1   albums (246  ) in 2600/4205	/Volumes/Music/Discog/albums/93/3988293
    Found 1   albums (257  ) in 2700/4205	/Volumes/Music/Discog/albums/93/4188393
    Found 2   albums (262  ) in 2800/4205	/Volumes/Music/Discog/albums/93/434193
    Found 0   albums (317  ) in 2900/4205	/Volumes/Music/Discog/albums/93/4545093
    Found 1   albums (323  ) in 3000/4205	/Volumes/Music/Discog/albums/93/472793
    Found 0   albums (328  ) in 3100/4205	/Volumes/Music/Discog/albums/93/4918093
    Found 12  albums (330  ) in 3200/4205	/Volumes/Music/Discog/albums/93/511193
    Found 1   albums (337  ) in 3300/4205	/Volumes/Music/Discog/albums/93/5283893
    Found 1   albums (342  ) in 3400/4205	/Volumes/Music/Discog/albums/93/5462493
    Found 1   albums (354  ) in 3500/4205	/Volumes/Music/Discog/albums/93/5825693
    Found 2   albums (357  ) in 3600/4205	/Volumes/Music/Discog/albums/93/61493
    Found 1   albums (378  ) in 3700/4205	/Volumes/Music/Discog/albums/93/6520293
    Found 1   albums 

    Found 0   albums (90   ) in 800/4167	/Volumes/Music/Discog/albums/95/1725595
    Found 0   albums (101  ) in 900/4167	/Volumes/Music/Discog/albums/95/182295
    Found 1   albums (109  ) in 1000/4167	/Volumes/Music/Discog/albums/95/1921395
    Found 4   albums (139  ) in 1100/4167	/Volumes/Music/Discog/albums/95/2023295
    Found 1   albums (144  ) in 1200/4167	/Volumes/Music/Discog/albums/95/2129195
    Found 7   albums (152  ) in 1300/4167	/Volumes/Music/Discog/albums/95/2246995
    Found 1   albums (163  ) in 1400/4167	/Volumes/Music/Discog/albums/95/2364295
    Found 4   albums (169  ) in 1500/4167	/Volumes/Music/Discog/albums/95/2474595
    Found 1   albums (180  ) in 1600/4167	/Volumes/Music/Discog/albums/95/2601195
    Found 1   albums (191  ) in 1700/4167	/Volumes/Music/Discog/albums/95/2724195
    Found 2   albums (195  ) in 1800/4167	/Volumes/Music/Discog/albums/95/2855695
    Found 1   albums (207  ) in 1900/4167	/Volumes/Music/Discog/albums/95/3001495
    Found 0   album

    Found 2   albums (134  ) in 900/3929	/Volumes/Music/Discog/albums/97/1822297
    Found 0   albums (142  ) in 1000/3929	/Volumes/Music/Discog/albums/97/1940097
    Found 1   albums (149  ) in 1100/3929	/Volumes/Music/Discog/albums/97/20697
    Found 1   albums (165  ) in 1200/3929	/Volumes/Music/Discog/albums/97/2188297
    Found 1   albums (180  ) in 1300/3929	/Volumes/Music/Discog/albums/97/2312697
    Found 2   albums (197  ) in 1400/3929	/Volumes/Music/Discog/albums/97/2466397
    Found 0   albums (206  ) in 1500/3929	/Volumes/Music/Discog/albums/97/2571597
    Found 1   albums (215  ) in 1600/3929	/Volumes/Music/Discog/albums/97/269697
    Found 2   albums (220  ) in 1700/3929	/Volumes/Music/Discog/albums/97/2818497
    Found 3   albums (233  ) in 1800/3929	/Volumes/Music/Discog/albums/97/2953397
    Found 1   albums (244  ) in 1900/3929	/Volumes/Music/Discog/albums/97/3119897
    Found 1   albums (252  ) in 2000/3929	/Volumes/Music/Discog/albums/97/3260397
    Found 1   albums